In [23]:
#Import 
import warnings; warnings.simplefilter('ignore') #for PCoA warnings
import pandas as pd
import numpy as np
import random
import itertools
#import data
from biom import load_table
from gneiss.util import match
#plotting
import seaborn as sns
import matplotlib.pyplot as plt
#MOCK data generation
from gneiss import util
from gneiss.util import band_diagonal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, poisson, multivariate_normal
from skbio.stats.composition import clr
#biplots
from sklearn.datasets import samples_generator as sg
from sklearn.cluster.bicluster import SpectralBiclustering
#from sklearn.metrics import consensus_score
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
#completions
from DEICODE.untangle import complete_matrix,machine_learning,biplot
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute,IterativeSVD,MICE,MatrixFactorization
#transforms 
from skbio.stats.composition import clr,ilr
#PCoA
from skbio import DistanceMatrix
from skbio.stats.ordination import pcoa
from scipy.spatial.distance import pdist, squareform
from skbio.stats import subsample_counts
from scipy import stats
#splits 
from sklearn.model_selection import train_test_split
# Intials
rand = np.random.RandomState(42)
fnts=12
# make it look nice
sns.set(font="monospace")
import warnings
warnings.filterwarnings("ignore")
from IPython.core.display import HTML
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('seaborn-darkgrid')
plt.style.use('seaborn-paper')
HTML("""<style>div.cell { /* Tunes the space between cells */margin-top:1em;margin-bottom:1em;}div.text_cell_render h1 { /* Main titles bigger, centered */
font-size: 3em;line-height:1.4em;text-align:center;}
div.text_cell_render h2 { /*  Parts names nearer from text */margin-bottom: -0.4em;}
div.text_cell_render { /* Customize text cells */font-family: 'Times New Roman';font-size:2.5em;line-height:1.4em;
padding-left:3em;padding-right:3em;}</style>""")
%matplotlib inline

# Import the datasets 

In [13]:
# import keyboard observation data
in_biom='data/benchmarking/keyboard.biom' #import biom file
table = load_table(in_biom)
read_filter = lambda val, id_, md: sum(val) > 0
table.filter(read_filter, axis='sample')
table.filter(read_filter, axis='observation')
otutabledf=table.to_dataframe()
otutabledf=otutabledf.T
otutabledf.drop_duplicates(inplace=True)

# Get OTU to taxa match
taxonomy=table.metadata_to_dataframe('observation')
taxonomy.columns=['kingdom', 'phylum', 'class', 'order', 
                             'family', 'genus', 'species']
taxonomy['taxonomy'] = taxonomy[taxonomy.columns].apply(lambda x: ';'.join(x), axis=1)

#mapping import 
map_file='data/benchmarking/keyboard.txt' #import metadata
mappingdf= pd.read_table('%s'%map_file, index_col=0,low_memory=False)
mappingdf=mappingdf.replace(np.nan,'Unknown', regex=True)
mappingdf.index=list(map(str,mappingdf.index))
mappingdf=mappingdf.astype(str)
mappingdf=mappingdf[~mappingdf.index.duplicated(keep='first')]

#match the tables
otu_keyboard,mapping_keyboard=match(otutabledf,mappingdf[mappingdf['host_subject_id'].isin(['M2','M3','M9'])])

In [4]:
# import 88 soils observation data
in_biom='data/benchmarking/88soils.biom' #import biom file
table = load_table(in_biom)
#read_filter = lambda val, id_, md: sum(val) > 0
read_filter_s = lambda val, id_, md: sum(val) > 500
table.filter(read_filter_s, axis='sample')
#table.filter(read_filter, axis='observation')
otutabledf=table.to_dataframe()
otutabledf=otutabledf.T
otutabledf.drop_duplicates(inplace=True)

# Get OTU to taxa match
taxonomy=table.metadata_to_dataframe('observation')
taxonomy.columns=['kingdom', 'phylum', 'class', 'order', 
                             'family', 'genus', 'species']
taxonomy['taxonomy'] = taxonomy[taxonomy.columns].apply(lambda x: ';'.join(x), axis=1)

#mapping import 
map_file='data/benchmarking/88soils.txt' #import metadata
mappingdf= pd.read_table('%s'%map_file, index_col=0,low_memory=False)
mappingdf=mappingdf.replace(np.nan,'Unknown', regex=True)
mappingdf.index=list(map(str,mappingdf.index))
mappingdf=mappingdf.astype(str)
mappingdf=mappingdf[~mappingdf.index.duplicated(keep='first')]

#match the tables
otu_soils,mapping_soils=match(otutabledf,mappingdf)

# Split the data and metadata into training (75%) and test (25%) 

In [22]:
# Warning: Takes a few min to run 

# split both datasets (75,25)
train_soils, test_soils = train_test_split(otu_soils,test_size=.25)
train_keyboard, test_keyboard = train_test_split(otu_keyboard,test_size=.25)

#make test and train metadata and match to other df for soils dataset 
train_soils,mapping_train_soils=match(train_soils,mapping_soils)
test_soils,mapping_test_soils=match(test_soils,mapping_soils)

#make test and train metadata and match to other df for Keyboard dataset 
train_keyboard,mapping_train_keyboard=match(train_keyboard,mapping_keyboard)
test_keyboard,mapping_test_keyboard=match(test_keyboard,mapping_keyboard)

# Test all completion methods on the training datasets 

## Output is saved as a list where completed_soils contains all soils datasets and completed_keyboard contains all keyboard datasets 

## The order for each list is :  [Psuedo, OpSpace, KNN, Soft Impute, Iterative SVD]

In [37]:
%%capture capt 
# to prevent a long output from fancy impute 

#save the output in an ordered list 
completed_soils=[]
completed_keyboard=[]

for X_train,la in zip([train_soils,train_keyboard],[completed_soils,completed_keyboard]):
        
    #psuedo count
    la.append(X_train+1)
    
    # OptSpace
    X_filled_opt=complete_matrix(X_train.as_matrix().copy(),rank=3,iteration=20,maxval=10,minval=.01)  
    la.append(pd.DataFrame(X_filled_opt,X_train.index,X_train.columns))
    
    # Make 0 NaN for fancy impute 
    X_noisefancy=X_train.as_matrix().copy()
    X_noisefancy[X_noisefancy==0]=np.nan
    
    # KNN
    X_filled_knn = KNN(k=20,verbose=False,min_value=.01,max_value=10).complete(X_noisefancy)
    la.append(pd.DataFrame(X_filled_knn,X_train.index,X_train.columns))
    
    #soft impute
    X_filled_softimpute = SoftImpute(verbose=False,min_value=.01,max_value=10).complete(X_noisefancy)
    la.append(pd.DataFrame(X_filled_softimpute,X_train.index,X_train.columns))
    
    # Iter SVD
    X_filled_iter=IterativeSVD(verbose=False,min_value=.01,max_value=1e1).complete(X_noisefancy)
    la.append(pd.DataFrame(X_filled_iter,X_train.index,X_train.columns))      

# Run Regression analysis and MSE

In [46]:
from gneiss.cluster import gradient_linkage
from gneiss.regression import ols